# Creating SQL Databases

### See Your Tables

* https://www.mysql.com/products/workbench/
* https://dbeaver.io/


### Schemas

https://dbdiagram.io/home

Creating SQL DBs in a few ways!! 

1. Pandas' `.to_sql` method
2. Manually via cursor and `INSERT INTO`
3. On an AWS RDB

In [1]:
import pandas as pd

In [2]:
# importing our sample data

customers = pd.read_csv('data/customers.csv')
order_details = pd.read_csv('data/orderdetails.csv')
orders = pd.read_csv('data/orders.csv')
products = pd.read_csv('data/products.csv')

display(customers.head(), order_details.head())

,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",NaN,Nantes,NaN,44000,France,1370.0,21000
1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,NaN,Las Vegas,NV,83030,USA,1166.0,71800
2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",NaN,Nantes,NaN,44000,France,1370.0,118200
4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,NaN,Stavern,NaN,4110,Norway,1504.0,81700


,orderNumber,productCode,quantityOrdered,priceEach,orderLineNumber
0,10100,S18_1749,30,136.00,3
1,10100,S18_2248,50,55.09,2
2,10100,S18_4409,22,75.46,4
3,10100,S24_3969,49,35.29,1
4,10101,S18_2325,25,108.06,4


## Using Pandas' `.to_sql()` method

In [3]:
from sqlalchemy import create_engine

# must start with 'sqlite:///' for a relative path
engine = create_engine('sqlite:///sales.db', echo=True) 
# echo determines whether actions are output

In [4]:
# as a function
def create_sql_table(df, table_name, engine):
    df.to_sql(table_name, con=engine, if_exists='replace')

In [5]:
create_sql_table(customers, 'customers', engine)

2021-02-04 14:10:25,739 INFO sqlalchemy.engine.base.Engine SELECT CAST('test plain returns' AS VARCHAR(60)) AS anon_1
2021-02-04 14:10:25,740 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:10:25,741 INFO sqlalchemy.engine.base.Engine SELECT CAST('test unicode returns' AS VARCHAR(60)) AS anon_1
2021-02-04 14:10:25,741 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:10:25,743 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("customers")
2021-02-04 14:10:25,744 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:10:25,746 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("customers")
2021-02-04 14:10:25,746 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:10:25,749 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE customers (
	"index" BIGINT, 
	"customerNumber" BIGINT, 
	"customerName" TEXT, 
	"contactLastName" TEXT, 
	"contactFirstName" TEXT, 
	phone TEXT, 
	"addressLine1" TEXT, 
	"addressLine2" TEXT, 
	city TEXT, 
	state TEXT, 
	"postalCode" TEXT, 
	country T

In [6]:
create_sql_table(order_details, 'order_details', engine)

2021-02-04 14:11:12,380 INFO sqlalchemy.engine.base.Engine PRAGMA main.table_info("order_details")
2021-02-04 14:11:12,380 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:11:12,381 INFO sqlalchemy.engine.base.Engine PRAGMA temp.table_info("order_details")
2021-02-04 14:11:12,382 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:11:12,385 INFO sqlalchemy.engine.base.Engine 
CREATE TABLE order_details (
	"index" BIGINT, 
	"orderNumber" BIGINT, 
	"productCode" TEXT, 
	"quantityOrdered" BIGINT, 
	"priceEach" FLOAT, 
	"orderLineNumber" BIGINT
)


2021-02-04 14:11:12,386 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:11:12,391 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-04 14:11:12,392 INFO sqlalchemy.engine.base.Engine CREATE INDEX ix_order_details_index ON order_details ("index")
2021-02-04 14:11:12,393 INFO sqlalchemy.engine.base.Engine ()
2021-02-04 14:11:12,399 INFO sqlalchemy.engine.base.Engine COMMIT
2021-02-04 14:11:12,404 INFO sqlalchemy.engine.base.Engine BEGIN (im

### Re-reading in our data

In [7]:
result = engine.execute('SELECT * FROM customers;')
pd.DataFrame(result.fetchall(), columns = result.keys())

2021-02-04 14:23:48,028 INFO sqlalchemy.engine.base.Engine SELECT * FROM customers;
2021-02-04 14:23:48,029 INFO sqlalchemy.engine.base.Engine ()


,index,customerNumber,customerName,contactLastName,contactFirstName,phone,addressLine1,addressLine2,city,state,postalCode,country,salesRepEmployeeNumber,creditLimit
0,0,103,Atelier graphique,Schmitt,Carine,40.32.2555,"54, rue Royale",None,Nantes,None,44000,France,1370.0,21000
1,1,112,Signal Gift Stores,King,Jean,7025551838,8489 Strong St.,None,Las Vegas,NV,83030,USA,1166.0,71800
2,2,114,"Australian Collectors, Co.",Ferguson,Peter,03 9520 4555,636 St Kilda Road,Level 3,Melbourne,Victoria,3004,Australia,1611.0,117300
3,3,119,La Rochelle Gifts,Labrune,Janine,40.67.8555,"67, rue des Cinquante Otages",None,Nantes,None,44000,France,1370.0,118200
4,4,121,Baane Mini Imports,Bergulfsen,Jonas,07-98 9555,Erling Skakkes gate 78,None,Stavern,None,4110,Norway,1504.0,81700
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
117,117,486,Motor Mint Distributors Inc.,Salazar,Rosa,2155559857,11328 Douglas Av.,None,Philadelphia,PA,71270,USA,1323.0,72600
118,118,487,Signal Collectibles Ltd.,Taylor,Sue,4155554312,2793 Furth Circle,None,Brisbane,CA,94217,USA,1165.0,60300
119,119,489,"Double Decker Gift Stores, Ltd",Smith,Thomas,(171) 555-7555,120 Hanover Sq.,None,London,None,WA1 1DP,UK,1501.0,43300
120,120,495,Diecast Collectables,Franco,Valarie,6175552555,6251 Ingle Ln.,None,Boston,MA,51003,USA,1188.0,85100


## AWS RDS (Relational Database System)

Note: this database just lives on AWS, you can do this same process on a local database!

Pros and cons of these vs `.to_sql()`:
* You can configure your DB more professionally
* Fairly manual process

Setting up a database instance: https://aws.amazon.com/getting-started/hands-on/create-mysql-db/ 

**It is extremely important that you read all the instructions in the linked page so everything is set up correctly!!!**

In [10]:
!pip install mysql-connector-python

In [11]:
import mysql.connector
from mysql.connector import errorcode

In [15]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

InterfaceError: 2003: Can't connect to MySQL server on 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com:3306' (10060 A connection attempt failed because the connected party did not properly respond after a period of time, or established connection failed because connected host has failed to respond)

In [ ]:
cursor.execute("DROP DATABASE sales;")

In [ ]:
# creating your database

db_name = 'sales'

def create_database(cursor, database):
    try:
        cursor.execute(
            "CREATE DATABASE {} DEFAULT CHARACTER SET 'utf8'".format(database))
    except mysql.connector.Error as err:
        print("Failed creating database: {}".format(err))
        exit(1)

In [ ]:
try:
    cursor.execute("USE {}".format(db_name))
except mysql.connector.Error as err:
    print("Database {} does not exist.".format(db_name))
    if err.errno == errorcode.ER_BAD_DB_ERROR:
        create_database(cursor, db_name)
        print("Database {} created successfully.".format(db_name))
        cnx.database = db_name
    else:
        print(err)
        exit(1)
        
cursor.close()
cnx.close()

In [ ]:
# creating tables

TABLES = {}
TABLES['customers'] = (
        "CREATE TABLE customers ("
        " customerNumber int(11) NOT NULL,"
        " customerName varchar(64) NOT NULL,"
        " contactLastName varchar(16) NOT NULL,"
        " contactFirstName varchar(16) NOT NULL,"
        " phone varchar(16) NOT NULL,"
        " addressLine1 varchar(64) NOT NULL,"
        " addressLine2 varchar(64) NOT NULL,"
        " city varchar(64) NOT NULL,"
        " state varchar(64) NOT NULL,"
        " postalCode int(10) NOT NULL,"
        " country varchar(64) NOT NULL,"
        " salesRepEmployeeNumber int(11) NOT NULL,"
        " creditLimit int(11) NOT NULL) ENGINE=InnoDB"
)



In [ ]:
customers.head()

In [ ]:
print(TABLES['customers'])

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

In [ ]:
for table_name in TABLES:
    table_description = TABLES[table_name]
    try:
        print("Creating table {}: ".format(table_name), end='')
        cursor.execute(table_description)
    except mysql.connector.Error as err:
        if err.errno == errorcode.ER_TABLE_EXISTS_ERROR:
            print("already exists.")
        else:
            print(err.msg)
    else:
        print("OK")

cursor.close()
cnx.close()

### Inserting Data

In [ ]:
customers.columns

In [ ]:
insert = ("INSERT INTO customers (customerNumber, "
"customerName, contactLastName, contactFirstName, phone, "
"addressLine1, addressLine2, city, state, postalCode, "
"country, salesRepEmployeeNumber, creditLimit) VALUES "
"(%s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s, %s)")

In [ ]:
insert

In [ ]:
customers = customers.dropna()
customers['salesRepEmployeeNumber'] = customers['salesRepEmployeeNumber'].astype(int)
customers = customers.applymap(str)

In [ ]:
customers.head()

In [ ]:
first = tuple(customers.iloc[0].values)
first

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

# this is the insert!
cursor.execute(insert, first)

# commiting changes and closing the connection
cnx.commit()
cursor.close()
cnx.close()

In [ ]:
data = [tuple(x) for x in customers.to_numpy()]
data[0]

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)

# cursor so we can interact with the db
cursor = cnx.cursor()

cursor.executemany(insert, data)

cnx.commit()
cursor.close()
cnx.close()

In [ ]:
# connecting to your database
cnx = mysql.connector.connect(
    host = 'test-db1.cpizore09tpz.us-east-2.rds.amazonaws.com',
    user = 'admin',
    passwd = 'testing123',
    database = 'sales'
)
cursor = cnx.cursor()

cursor.execute("SELECT * FROM customers;")
pd.DataFrame(cursor.fetchall(), columns=[x[0] for x in cursor.description])

You can either use the above method to read in your data, or use `pd.read_sql` (e